In [4]:
from tkinter import *
from setup import *
from random import random
import pandas as pd
import numpy as np
import glob
import os

###################################GLOBALS
CURRENT_SESSION = 1
color="purple"
default_color="white"
ROWS = 8
COLUMNS = 6
NUMJUDGES = 4
#############################################
df = pd.read_csv("gb_id_projects.csv")
# display(df)
pdf = df[['session', 'location','PRESENTATION TYPE','NAME','GUIDEBOOK ID']]
pdf = pdf.loc[pdf['PRESENTATION TYPE']=='poster']
device_df = pd.read_csv('judge_devices.csv')
judge_devices = list(device_df['DEVICE ID'])
poster_sessions = []

for i in range(4):
    tf = pdf[pdf['session'] == i+1]
    poster_sessions.append(tf.sort_values('location').reset_index(drop=True))
    
# display((poster_sessions[3].loc[poster_sessions[3]['GUIDEBOOK ID']=='8786706'])['location'])
poster_list = poster_sessions[CURRENT_SESSION-1]
poster_num =len(poster_list)
print('Num Posters: ' + str(poster_num))
print('Rows: ' + str(ROWS) + ', Cols: ' + str(COLUMNS))
# display(poster_list)

button_list = []
for i in range(ROWS*COLUMNS):
    button_list.append([default_color, NUMJUDGES])

buttons = []   
location_list = []
for i in range(ROWS):
    for j in range(COLUMNS):
        location_list.append(str(i+1)+str(chr(ord('a')+j)))
# print(location_list)
def main():
    window = Tk()
    Grid.rowconfigure(window, 0, weight=1)
    Grid.columnconfigure(window, 0, weight=1)
    frame=Frame(window)
    frame.grid(row=0, column=0, sticky=N+S+E+W)
    for x in range(ROWS):
        Grid.rowconfigure(frame, x, weight=1)
        for y in range(COLUMNS):
            i = x*COLUMNS+y
            Grid.columnconfigure(frame, y, weight=1)
            b = Button(frame,text = str(location_list[x*COLUMNS+y]) + '\n' + 'NEEDS ' + str(NUMJUDGES), bg=default_color, activebackground=default_color)
            b.grid(column=y, row=x)
            b["command"] = lambda b=b, x=x, y=y: click(b, x, y)
            b.grid(row=x, column=y, sticky=N+S+E+W)
            buttons.append(b)
    Grid.columnconfigure(frame, COLUMNS, weight=1)
    Grid.rowconfigure(frame, int(ROWS/2), weight=1)
    b = Button(frame, text = 'update', bg = default_color, activebackground = default_color)
    b["command"] = lambda b=b : update()
    b.grid(column = COLUMNS, row = int(ROWS/2), sticky = N+S+E+W)
    return window

def convert_num(id_list):
    num_list = []
    for x in id_list:
        num_list.append(poster_list.index(x))
    return num_list
    

def click(button,x,y):
    i = x*COLUMNS+y
    num = single_count(i)
#     print(x,y,i)
    if button_list[i][0] is color:
        button_list[i][0] = default_color
        button["bg"] = default_color
        button["activebackground"] = default_color
        button["text"] =str(location_list[x*COLUMNS+y]) + '\n' + 'NEEDS ' + str(NUMJUDGES-num)
    elif button_list[i][0] is default_color:
        button_list[i][0] = color
        button["bg"] = color
        button["activebackground"] = color
        button["text"] = str(location_list[x*COLUMNS+y]) + '\n'+ 'DISPATCHED - ' + str(NUMJUDGES-num)
    if num >= 4:
        button_list[i][0] = "grey"
        button["bg"] = "grey"
        button["activebackground"] = "grey"
        button["text"] = str(location_list[x*COLUMNS+y]) + '\n'+ 'DONE'
def oupdate():
    list_of_files = glob.glob('./inc_guidebook/*') # * means all if need specific format then *.csv
    latest_file = max(list_of_files, key=os.path.getctime)
    df = pd.read_csv(latest_file, names = ['Judge_ID','Poster_ID','Presenter', 'Question_ID', 'Question', 'Response','Date'])
    ndf = df.iloc[guidebook_loc,:]
    change_list = ndf['Poster_ID']
    change_unique = list(set(change_list))
    change_convert = convert_num(change_unique)
    change_counts = [[x,int(change_list.count(x)/5)] for x in change_unique]

    
    change_list = np.asarray(np.random.choice(poster_num,size=10, replace=False),dtype=np.int)
    change_list= change_list.reshape(10)
#     print(len(buttons))
#     print(random_list)
    for i in change_list:
#         print(i)
        button_column = int(i%COLUMNS)
        button_row = int((i-button_column)/COLUMNS)
#         print(button_row,button_column,i)
        if button_list[i][1] <= 0:
            button_list[i][0] = "grey"
            buttons[i]["bg"] = "grey"
            buttons[i]["activebackground"] = "grey"
            buttons[i]["text"] = str(button_row)+str(button_column)+ '- DONE'
        elif button_list[0] is color:
            button_list[i][0] = default_color
            buttons[i]["bg"] = default_color
            buttons[i]["activebackground"] = default_color
            buttons[i]["text"] = str(x)+str(y)+ '- NEEDS ' + str(button_list[i][1]) + '\n'+str(poster_list[i])
            
        else:
            buttons[i]["text"]=str(button_row)+str(button_column)+ '- NEEDS ' + str(button_list[i][1]) + '\n'+str(poster_list[i])
            button_list[i][0] = default_color
            buttons[i]["bg"] = default_color
            buttons[i]["activebackground"] = default_color

def update():
    list_of_files = glob.glob('./session_update/*') # * means all if need specific format then *.csv
    latest_file = max(list_of_files, key=os.path.getctime)
    df = pd.read_csv(latest_file, names = ['Judge_ID','Poster_ID','Presenter', 'Question_ID', 'Question', 'Response','Date'])
    pdf = df.loc[df['Question']=='Judges Password']
    mdf = df[df['Response'].str.lower()=='score']
    ldf = df[df['Judge_ID'].str.lower().isin(judge_devices)]
    ndf = pd.merge(mdf,ldf,how = 'outer')
    grouped_df = ndf.groupby('Poster_ID').apply(lambda g: len(g['Judge_ID'].unique()))
#     display(grouped_df)
    display(grouped_df)
    update_list = list(grouped_df.index)
#     print(update_list)
    for project in update_list:
        if int(project) >= 8000000:
            print(project)
            num = grouped_df.loc[project]
            location = poster_list.loc[poster_list['GUIDEBOOK ID']==project]
            display(location)
            location = location['location'].to_string(index=False)
            display(location)
            i = location_list.index(location)
            button_column = int(i%COLUMNS)
            button_row = int((i-button_column)/COLUMNS)
    #         print(button_row,button_column,i)
            if num >= 4:
                button_list[i][0] = "grey"
                buttons[i]["bg"] = "grey"
                buttons[i]["activebackground"] = "grey"
                buttons[i]["text"] = str(location_list[COLUMNS*button_row+button_column]) + '\n'+ 'DONE'
            elif button_list[0] is color:
                button_list[i][0] = default_color
                buttons[i]["bg"] = default_color
                buttons[i]["activebackground"] = default_color
                buttons[i]["text"] = str(location_list[COLUMNS*button_row+button_column]) + '\n' + 'NEEDS ' + str(NUMJUDGES-num)

            else:
                buttons[i]["text"]=str(location_list[COLUMNS*button_row+button_column]) + '\n' + 'NEEDS ' + str(NUMJUDGES-num)
                button_list[i][0] = default_color
                buttons[i]["bg"] = default_color
                buttons[i]["activebackground"] = default_color        
#     display(grouped_df)
    
def single_count(i):
    list_of_files = glob.glob('./session_update/*') # * means all if need specific format then *.csv
    latest_file = max(list_of_files, key=os.path.getctime)
    df = pd.read_csv(latest_file, names = ['Judge_ID','Poster_ID','Presenter', 'Question_ID', 'Question', 'Response','Date'])
    pdf = df.loc[df['Question']=='Judges Password']
    mdf = df[df['Response'].str.lower()=='score']
    ldf = df[df['Judge_ID'].str.lower().isin(judge_devices)]
    ndf = pd.merge(mdf,ldf,how = 'outer')
#     display(ndf)
    grouped_df = ndf.groupby('Poster_ID').apply(lambda g: len(g['Judge_ID'].unique()))
    item = poster_list.loc[poster_list['location']==location_list[i]]
#     print('HERE IT IS')
#     display(poster_list)
#     print(item['GUIDEBOOK ID'].to_string(index=False))
#     display(grouped_df)
    if item['GUIDEBOOK ID'].to_string(index=False) in list(grouped_df.index):
        return grouped_df.loc[item['GUIDEBOOK ID'].to_string(index=False)]
    else:
        return 0
    
    
w = main()
mainloop()


Num Posters: 47
Rows: 8, Cols: 6


,0
Poster_ID,
8784599,1
8786577,3
8786582,1
8786584,1
8786588,1
8786591,2
8786592,1
8786595,2
8786598,2


8784599


,session,location,PRESENTATION TYPE,NAME,GUIDEBOOK ID


'Series([], )'

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\sw_hutyra\AppData\Local\Continuum\Anaconda3\lib\tkinter\__init__.py", line 1699, in __call__
    return self.func(*args)
  File "<ipython-input-4-ef70487fa14e>", line 65, in <lambda>
    b["command"] = lambda b=b : update()
  File "<ipython-input-4-ef70487fa14e>", line 153, in update
    i = location_list.index(location)
ValueError: 'Series([], )' is not in list
